In [ ]:
import ipyleaflet as ileaflet
import ipywidgets as widgets
from IPython.display import display, clear_output
import geopandas as gpd
import pandas as pd
import os

# Load the GeoJSON data
geojson_path = '../output/final_data_ne.geojson'
poultry_barns = gpd.read_file(geojson_path)

if os.path.exists('../output/checked_barns_indices.txt'):
    with open('../output/checked_barns_indices.txt', 'r') as file:
        lines = file.readlines()
    checked_indices = [(int(line.strip().split(',')[0]), int(line.strip().split(',')[1])) for line in lines]
    print(checked_indices)
else:
    # Initialize a list to keep track of checked indices
    checked_indices = []

# Filter out already false positives
poultry_barns = poultry_barns[(poultry_barns['false_positive'] != 1)]

already_check_lst = [i[0] for i in checked_indices]

poultry_barns = poultry_barns[~poultry_barns.index.isin(already_check_lst)]

# Create and display the map initially
m = ileaflet.Map(zoom=18, basemap=ileaflet.basemaps.Esri.WorldImagery)

def handle_classification(button, result):
    index = button.index
    centroid = button.centroid
    classification = button.description

    # Update the list with checked index and save to file
    checked_indices.append((index, result))
    with open('../output/checked_barns_indices.txt', 'a') as file:
        file.write(f'{index}, {result}\n')
    
    clear_output(wait=True)
    print(f"Barns remaining: {len(poultry_barns) - len(checked_indices)}")

    # Move to the next location
    next_index = index + 1
    if next_index < len(poultry_barns):
        display_location(next_index)
    else:
        print("All locations have been reviewed.")

# Display function for each location
def display_location(index):
    feature = poultry_barns.iloc[index]
    coords = feature.geometry.exterior.coords.xy
    centroid = (sum(coords[1]) / len(coords[1]), sum(coords[0]) / len(coords[0]))

    # Update map
    m.center = centroid
    m.layers = [m.layers[0]]

    polygon = ileaflet.Polygon(locations=[(y, x) for x, y in zip(*coords)], color="red", fill_color="red")
    m.add_layer(polygon)

    # Buttons with updated layout properties for size and alignment
    button_style = widgets.Layout(width='200px', height='50px', margin='5px')
    btn_true_positive = widgets.Button(description='True Positive', button_style='success', layout=button_style)
    btn_true_negative = widgets.Button(description='True Negative', button_style='warning', layout=button_style)
    btn_true_positive.on_click(lambda b: handle_classification(b, 1))
    btn_true_negative.on_click(lambda b: handle_classification(b, 0))
    btn_true_positive.index = index
    btn_true_negative.index = index
    btn_true_positive.centroid = centroid
    btn_true_negative.centroid = centroid

    # Vertical box for buttons aligned to the center-right of the map
    button_box = widgets.VBox([btn_true_positive, btn_true_negative], layout=widgets.Layout(align_items='center'))
    display(widgets.HBox([m, button_box], layout=widgets.Layout(align_items='center', justify_content='space-between')))

# Start the process with the first location if any are left
if not poultry_barns.empty:
    display_location(poultry_barns.index[0])
else:
    print("All locations have been reviewed.")


In [1]:
import ipyleaflet as ileaflet
import ipywidgets as widgets
from IPython.display import display, clear_output
import geopandas as gpd
import pandas as pd
import os

# Load the GeoJSON data
geojson_path = '../output/final_data_ne.geojson'
poultry_barns = gpd.read_file(geojson_path)

if os.path.exists('../output/checked_barns_indices.txt'):
    with open('../output/checked_barns_indices.txt', 'r') as file:
        lines = file.readlines()
    checked_indices = [(int(line.strip().split(',')[0]), int(line.strip().split(',')[1])) for line in lines]
else:
    checked_indices = []  # Initialize if the file doesn't exist

poultry_barns = poultry_barns[poultry_barns['false_positive'] != 1]
ori_length = len(poultry_barns)
already_checked_list = [i[0] for i in checked_indices]
poultry_barns = poultry_barns[~poultry_barns.index.isin(already_checked_list)]



def handle_classification(button, result):
    index = button.index
    checked_indices.append((index, result))
    with open('../output/checked_barns_indices.txt', 'a') as file:
        file.write(f'{index}, {result}\n')
    
    clear_output(wait=True)
    print(f"Barns remaining: {ori_length - len(checked_indices)}")
    next_index = index + 1
    if next_index < len(poultry_barns):
        display_location(next_index)
    else:
        print("All locations have been reviewed.")

def display_location(index):
    m = ileaflet.Map(zoom=18, basemap=ileaflet.basemaps.Esri.WorldImagery)
    feature = poultry_barns.iloc[index]
    coords = feature.geometry.exterior.coords.xy
    centroid = (sum(coords[1]) / len(coords[1]), sum(coords[0]) / len(coords[0]))
    m.center = centroid
    m.layers = [m.layers[0]]  # Keep the base map only

    polygon = ileaflet.Polygon(locations=[(y, x) for x, y in zip(*coords)], color="red", fill_color="red")
    m.add_layer(polygon)

    button_style = widgets.Layout(width='200px', height='50px', margin='5px')
    btn_true_positive = widgets.Button(description='True Positive', button_style='success', layout=button_style)
    btn_true_negative = widgets.Button(description='True Negative', button_style='warning', layout=button_style)
    btn_true_positive.on_click(lambda b: handle_classification(b, 1))
    btn_true_negative.on_click(lambda b: handle_classification(b, 0))
    btn_true_positive.index = index
    btn_true_negative.index = index
    btn_true_positive.centroid = centroid
    btn_true_negative.centroid = centroid

    button_box = widgets.VBox([btn_true_positive, btn_true_negative], layout=widgets.Layout(align_items='center'))
    display(widgets.HBox([m, button_box], layout=widgets.Layout(align_items='center', justify_content='space-between')))

if not poultry_barns.empty:
    display_location(poultry_barns.index[0])
else:
    print("All locations have been reviewed.")


Barns remaining: 4226
